# Extraction et similarité entre CV et Offre d'emploie

In [74]:
CV={'sexe': ['Femme '], 'formation': ['Master M2 (International of Computer Science)','Deux années de classes préparatoires intégrées et deux années cycle ingénieur en Intelligence Artificielle et Génie Informatique.','Une année de spécialisation en Mathématiques-Informatique et en Physique.','Baccalauréat option physique et chimie.'], 'localisation': ['Clermont-Ferrand'], 'competences': ['Python', 'R', 'Matlab', 'ML', 'DL', 'PCA', 'NLP', 'LLMs', 'langchain', 'Django', 'Flask', 'Streamlit', 'Flutter', 'React JS', 'Mysql', 'SQLite', 'Oracle', 'Mongodb', 'Git', 'Power BI', 'Jupyter', 'Google Colab', 'Tensorboard', 'MLflow '], 'experiences': ["2 ans"]}
# CV={'sexe': ['Femme '], 'formation': ['BSc (Hons) Business Information Technology & Human Resource Management', 'Nuneaton University 2005 - 2008', 'A levels: Maths (A) English (B) Technology (B) Science (C) Coventry Central College 2003 - 2005', 'ITEC level 2 Certificate in Human Resources'], 'localisation': ['Coventry'], 'competences': ['Recrutement', 'relations avec les employés', 'administration de la paie', 'gestion de la performance', "législation sur l\\'emploi", 'rédaction de rapports financiers', 'égalité des chances', 'gestion des absences', 'administration', 'communication', 'planification', 'prise de décision'], 'experiences': ['15 années']}
Offre={
  "sexe": [
  ],
  "formation": [
    "Diplôme d\\'ingénieur",
    "Master 2 "
  ],
  "localisation": [
    "Paris 15e Arrondissement"
  ],
  "competences": [
    "Python",
    "pandas",
    "numpy",
    "scikit-learn",
    "Machine Learning",
    "Deep Learning",
    "NLP",
    "concepts statistiques",
    "PowerShell",
    "Bash",
    "Git",
    "DevSecOps",
    "orchestration de conteneurs",
    "systèmes d\\'exploitation (Windows",
    "Linux)"
  ],
  "experiences": [
    "3 ans"
  ]
}

In [75]:
CV

{'sexe': ['Femme '],
 'formation': ['Master M2 (International of Computer Science)',
  'Deux années de classes préparatoires intégrées et deux années cycle ingénieur en Intelligence Artificielle et Génie Informatique.',
  'Une année de spécialisation en Mathématiques-Informatique et en Physique.',
  'Baccalauréat option physique et chimie.'],
 'localisation': ['Clermont-Ferrand'],
 'competences': ['Python',
  'R',
  'Matlab',
  'ML',
  'DL',
  'PCA',
  'NLP',
  'LLMs',
  'langchain',
  'Django',
  'Flask',
  'Streamlit',
  'Flutter',
  'React JS',
  'Mysql',
  'SQLite',
  'Oracle',
  'Mongodb',
  'Git',
  'Power BI',
  'Jupyter',
  'Google Colab',
  'Tensorboard',
  'MLflow '],
 'experiences': ['2 ans']}

In [76]:
Offre

{'sexe': [],
 'formation': ["Diplôme d\\'ingénieur", 'Master 2 '],
 'localisation': ['Paris 15e Arrondissement'],
 'competences': ['Python',
  'pandas',
  'numpy',
  'scikit-learn',
  'Machine Learning',
  'Deep Learning',
  'NLP',
  'concepts statistiques',
  'PowerShell',
  'Bash',
  'Git',
  'DevSecOps',
  'orchestration de conteneurs',
  "systèmes d\\'exploitation (Windows",
  'Linux)'],
 'experiences': ['3 ans']}

### **Problème: Ajout d'information supplémentaire (exemple Master M2 International of Computer Science et Master M2)**

In [77]:
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import spacy

nltk.download('wordnet')
nltk.download('omw-1.4')

# Load the French language model in spaCy
# !python -m spacy download fr_core_news_sm

# nlp = spacy.load('fr_core_news_sm')

lemmatizer = WordNetLemmatizer()

abbreviation_dict = {
    'ml': 'machine learning',
    'f':"femme",
    'h': "homme",
    'dl': 'deep learning',
    'ai': 'artificial intelligence',
    'rh':"ressources humaines",
    'm2': "master 2",
    'ing': "ingénieur"

}

degrees=["deug","deust","licence","master","doctorat","ingénieur"]

years_suffix=["an","année","ans","années"]


def lemmatize_text(text):
    words = text.split()
    return ' '.join([lemmatizer.lemmatize(word) for word in words])

def array_to_text(arr):
    return ' '.join(arr)

def expand_abbreviations(text):
    words = text.split()
    expanded_text = ' '.join([abbreviation_dict.get(word, word) for word in words])
    return expanded_text

def preprocess_text(text):
    text = text.lower()
    # text = re.sub(r'[\\]', '', text)
    text = re.sub(r"[^\w\s']", '', text)

    #Méthode 1:
    text = re.sub(r"\b(le|la|les|de|des|du|d'|l')\b", ' ', text)

    #Méthode 2:
    # Parse the text using spaCy
    # doc = nlp(text)

    # Remove French articles
    # articles = {"le", "la", "les", "de", "des", "du", "d'", "l'", "un", "une"}
    # text = ' '.join([token.text for token in doc if token.text not in articles])

    text = lemmatize_text(text)
    text = expand_abbreviations(text)
    return text

def compute_similarity(text1, text2):
    # Assure que les deux textes ne sont pas vides
    if not text1.strip() or not text2.strip():
        return 0  # ou une autre valeur par défaut si l'un des textes est vide

    # Configuration de TfidfVectorizer pour inclure des chiffres
    vectorizer = TfidfVectorizer(analyzer='word', token_pattern=r'\b\w+\b')
    vectors = vectorizer.fit_transform([text1, text2])
    return cosine_similarity(vectors[0:1], vectors[1:2])[0][0]

# Rendre les éléments des listes en minuscules et les lemmatiser
for key in CV:
    CV[key] = [preprocess_text(element) for element in CV[key]]
for key in Offre:
    Offre[key] = [preprocess_text(element)  for element in Offre[key]]

# Conversion des dictionnaires en texte
#sexe
if CV['sexe']:
  sexe_CV = CV['sexe'][0]
if Offre['sexe']:
  sexe_offre = Offre['sexe'][0]
else:
  sexe_offre=""

#formation
filtered_Offre_formation=([elem for item in Offre['formation'] for elem in item.split() ])
filtered_CV_formation = ' '.join(set([elem for item in CV['formation'] for elem in item.split() if elem in filtered_Offre_formation]))
formation_offre = array_to_text(filtered_Offre_formation)

#competences
filtered_CV_competence = ' '.join([item for item in CV['competences'] if item in Offre['competences']])
competences_offre = array_to_text(Offre['competences'])

#experience
filtered_Offre_experience = ' '.join([
    str(0) if elem == 'débutant' else
    str(2) if elem == 'junior' else
    str(5) if elem == 'senior' else
    elem
    for item in Offre['experiences']
    for elem in item.split()
    if elem not in years_suffix
])

# filtered_CV_experience = ' '.join(set([elem for item in CV['experiences'] for elem in item.split() if elem in filtered_Offre_experience]))
filtered_CV_experience = ' '.join(set([elem for item in CV['experiences'] for elem in item.split() if elem not in years_suffix]))
experience_offre = array_to_text(filtered_Offre_experience)

# Calcul de la cosine similarity
if sexe_CV and sexe_offre:
  cosine_sim_sexe = compute_similarity(sexe_CV,sexe_offre)
cosine_sim_formation = compute_similarity(filtered_CV_formation,formation_offre)
cosine_sim_competences = compute_similarity(filtered_CV_competence,competences_offre)
cosine_sim_experiences = compute_similarity(filtered_CV_experience,experience_offre)

if cosine_sim_experiences == 1 or (filtered_CV_experience and filtered_Offre_experience and float(filtered_CV_experience) >= float(filtered_Offre_experience)):
  cosine_sim_experiences=1
else:
  cosine_sim_experiences=0

if sexe_offre!='femme' and sexe_offre!='homme':
  cosine_sim_sexe=1

if cosine_sim_competences==0:
  cosine_sim_experiences=0

cosine_sim=(cosine_sim_sexe+cosine_sim_formation+cosine_sim_competences+cosine_sim_experiences)/4

if cosine_sim_sexe==0:
  cosine_sim=0

print(f"Similarité Cosine : {cosine_sim}")


Similarité Cosine : 0.5680073564323089


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
